In [1]:
from pathlib import Path
from scipy.io import loadmat
import numpy as np
import pandas as pd

In [2]:
def get_start_time(H, row):
    """Infer a bout's start time from the previous bout's end time.

    Parameters
    ----------
    H: DataFrame, (n_bouts, ?)
        Hypogram in Visbrain format with 'start_time'.
    row: Series
        A row from `H`, representing the bout that you want the start time of.

    Returns
    -------
    start_time: float
        The start time of the bout from `row`.
    """
    if row.name == 0:
        start_time = 0.0
    else:
        start_time = H.loc[row.name - 1].end_time

    return start_time


def load_visbrain_hypnogram(path):
    """Load a Visbrain-formatted hypngoram.

    Parameters
    ----------
    path: pathlib.Path
        The path to the hypnogram.

    Returns:
    --------
    H: DataFrame
        The loaded hypnogram.
    """
    H = pd.read_csv(path, sep="\t", names=["state", "end_time"], skiprows=1)
    H["start_time"] = H.apply(lambda row: get_start_time(H, row), axis=1)
    H["duration"] = H.apply(lambda row: row.end_time - row.start_time, axis=1)

    return H

In [6]:
def get_state(t, hypnogram):
    bout = hypnogram[np.logical_and(hypnogram.start_time <= t, hypnogram.end_time >= t)]
    assert len(bout) == 1, f"More than one state found for time {t}"
    return bout.state.squeeze()

In [3]:
stim_times_path = Path('/Volumes/neuropixel/Data/CNPIX8-Allan/3-5-2021/3-5-2021_g3/stim_times.mat')
hypnogram_path = Path('/Volumes/neuropixel/Data/CNPIX8-Allan/3-5-2021/3-5-2021_g3/hypno.txt')

In [4]:
hypnogram = load_visbrain_hypnogram(hypnogram_path)
m = loadmat(stim_times_path, squeeze_me=True)

In [5]:
stim_times = m['sglx_stim_times']
stim_times = stim_times[np.logical_and(stim_times >= 0, stim_times <= hypnogram.end_time.max())]
stim_states = np.vectorize(lambda t: get_state(t, hypnogram))(stim_times)
df = pd.DataFrame({'sglx_stim_time': stim_times, 'stim_state': stim_states})

In [9]:
df.to_csv('stim_times.csv', index=False)